# Analysis of dataset to assign label

The purpose of this subproject is ; in a simple and easy way; assign a better label for classification in kind of disaster and whether or not it is a joke.

As dataset is pre-labeled from human team; i will use this features like basis and guide for this process.

## Preparing enviroment to work

In [1]:
!pip install boto3
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 69.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from boto3 import Session

In [3]:
disaster_db_df = pd.read_csv("/database.csv", encoding="latin9")
disaster_db_df.head(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
0,778243823,True,golden,156,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just happened a terrible car crash,1.0,NaN
1,778243824,True,golden,152,NaN,Relevant,1.0000,Relevant,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,13.0,NaN
2,778243825,True,golden,137,NaN,Relevant,1.0000,Relevant,NaN,NaN,"Heard about #earthquake is different cities, s...",14.0,NaN
3,778243826,True,golden,136,NaN,Relevant,0.9603,Relevant,NaN,NaN,"there is a forest fire at spot pond, geese are...",15.0,NaN
4,778243827,True,golden,138,NaN,Relevant,1.0000,Relevant,NaN,NaN,Forest fire near La Ronge Sask. Canada,16.0,NaN


Based in data sample, i think the `choose_one`, `choose_one:confidence`, `choosen_one_gold`, `text` and `tweetid` are important for the analysis; i need to check if my hypothesis is true.

---



# Checking the mentioned features are relevant or not

### **`Choose_one` field**

In [4]:
disaster_db_df['choose_one'].value_counts()

Not Relevant    6187
Relevant        4673
Can't Decide      16
Name: choose_one, dtype: int64

In [5]:
disaster_db_df['choose_one'].isnull().value_counts()

False    10876
Name: choose_one, dtype: int64

That's ready to use ✅

### **`choose_one_gold` field**

In [6]:
disaster_db_df['choose_one_gold'].value_counts()

Relevant                      57
Not Relevant                  24
Relevant\nCan't Decide         4
Not Relevant\nCan't Decide     2
Name: choose_one_gold, dtype: int64

In [7]:
disaster_db_df['choose_one_gold'].isnull().value_counts()

True     10789
False       87
Name: choose_one_gold, dtype: int64

***QUESTION #1: Can i replace it with `choose_one` field?***

In [8]:
disaster_db_df.replace(np.nan, None)
disaster_db_df["choose_one_gold_v2"] = disaster_db_df["choose_one_gold"].apply(
    lambda x: x.split("\n")[-1] if type(x) is str else None
)
disaster_db_df["choose_one_gold_v2"].value_counts()

Relevant        57
Not Relevant    24
Can't Decide     6
Name: choose_one_gold_v2, dtype: int64

In [9]:
disaster_db_df["is_choose_one_gold_with_choose_one"] = disaster_db_df.apply( 
    lambda x: True if x["choose_one_gold_v2"] == x["choose_one_gold"] else False, 
    axis=1
  )
selected_choose_one_gold_df = disaster_db_df[ disaster_db_df["choose_one_gold_v2"].isin(["Relevant", "Not Relevant", "Can't Decide"])]
selected_choose_one_gold_df["is_choose_one_gold_with_choose_one"].value_counts()

True     81
False     6
Name: is_choose_one_gold_with_choose_one, dtype: int64

In [10]:
selected_choose_one_gold_df = selected_choose_one_gold_df[selected_choose_one_gold_df["is_choose_one_gold_with_choose_one"].isin([False])]
selected_choose_one_gold_df[["choose_one_gold_v2", "choose_one", "choose_one_gold", "is_choose_one_gold_with_choose_one"]]

,choose_one_gold_v2,choose_one,choose_one_gold,is_choose_one_gold_with_choose_one
9,Can't Decide,Relevant,Relevant\nCan't Decide,False
13,Can't Decide,Relevant,Relevant\nCan't Decide,False
15,Can't Decide,Relevant,Relevant\nCan't Decide,False
19,Can't Decide,Relevant,Relevant\nCan't Decide,False
42,Can't Decide,Not Relevant,Not Relevant\nCan't Decide,False
44,Can't Decide,Not Relevant,Not Relevant\nCan't Decide,False


In previous table, i see that all non-null value of `choose_one_gold` field are same with `choose_one` field.

Ok, i ***won't use this field*** because it have several null value and i have more value from `choose_one` ❌

### **`choose_one:confidence` field**

In [11]:
disaster_db_df['choose_one:confidence'].isnull().value_counts()

False    10876
Name: choose_one:confidence, dtype: int64

In [13]:
def assign_range_value(x):
  if 0.0 < x <= 0.5: return "0.0 - 0.5"
  elif 0.5 < x <= 0.75: return "0.5 - 0.75"
  elif 0.75 < x < 0.9: return "0.75 - 0.9"
  elif 0.9 < x <= 1.0: return "0.9 - 1.0"
  else: return "> 1.0"

disaster_db_df["range_confidence"] = disaster_db_df["choose_one:confidence"].apply(lambda x: assign_range_value(x))
disaster_db_df['range_confidence'].value_counts()

0.9 - 1.0     5066
0.75 - 0.9    3075
0.5 - 0.75    2597
0.0 - 0.5      138
Name: range_confidence, dtype: int64

i can see that 74.85% of tweets are trustworthy in with its value: That's good 😀. But the question is: **How is thise proportion is distribuite over `choosen_one` field?**

In [16]:
Relevant_group_df = disaster_db_df[disaster_db_df["choose_one"] == "Relevant"]
Relevant_group_df['range_confidence'].value_counts()

0.9 - 1.0     2273
0.75 - 0.9    1172
0.5 - 0.75    1148
0.0 - 0.5       80
Name: range_confidence, dtype: int64

In [17]:
Relevant_group_df = disaster_db_df[disaster_db_df["choose_one"] == "Not Relevant"]
Relevant_group_df['range_confidence'].value_counts()

0.9 - 1.0     2793
0.75 - 0.9    1903
0.5 - 0.75    1445
0.0 - 0.5       46
Name: range_confidence, dtype: int64

In [18]:
Relevant_group_df = disaster_db_df[disaster_db_df["choose_one"] == "Can't Decide"]
Relevant_group_df['range_confidence'].value_counts()

0.0 - 0.5     12
0.5 - 0.75     4
Name: range_confidence, dtype: int64

⚡ **Idea**: I can see the `Can't Decide` value in `choose_one` field is irrelevant for the amount, but i see that `0.5-0.75` range of other values of `choose_one` field can be renamed in `Can't Decide` value to get more amount: from 16 to 2609; that's good comparing other group amount 4.696(`Not Relevant`) and 3445(`Relevant`).

I won't use any value from `choose_one` field with `0.0-5.0` range because these values are irrelevant: they are not trustworthy.

### **`tweetid` and `text` fields**

In [39]:
disaster_db_df['tweetid'].isnull().value_counts()

False    10876
Name: tweetid, dtype: int64

In [40]:
disaster_db_df.astype({'tweetid': 'str'}).dtypes

_unit_id                                int64
_golden                                  bool
_unit_state                            object
_trusted_judgments                      int64
_last_judgment_at                      object
choose_one                             object
choose_one:confidence                 float64
choose_one_gold                        object
keyword                                object
location                               object
text                                   object
tweetid                                object
userid                                float64
choose_one_gold_v2                     object
is_choose_one_gold_with_choose_one       bool
range_confidence                       object
dtype: object

In [41]:
unique_tweet_id = set(disaster_db_df['tweetid'])
list_tweet_id = list(disaster_db_df['tweetid'])
print("unique value are %s from %s" % (len(unique_tweet_id), len(list_tweet_id)))

unique value are 1243 from 10876


In [47]:
disaster_db_df['tweetid'].value_counts()

6.290840e+17    197
6.290830e+17    190
6.290990e+17    166
6.290800e+17    157
6.290820e+17    152
               ... 
6.260600e+17      1
6.260750e+17      1
6.262370e+17      1
6.276670e+17      1
6.585000e+03      1
Name: tweetid, Length: 1243, dtype: int64

In [50]:
disaster_db_df["is_tweet_id_duplicated"] = disaster_db_df.duplicated(subset=['tweetid'])
duplicated_tweetit_db_df = disaster_db_df[disaster_db_df["is_tweet_id_duplicated"] == True]
duplicated_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)
duplicated_tweetit_db_df[["tweetid", "text"]].head(20)

<ipython-input-50-2d003273006d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated_tweetit_db_df.sort_values(by = 'tweetid', inplace=True)


,tweetid,text
10872,4.234000e+03,Police investigating after an e-bike collided ...
9066,6.260620e+17,Investigators say a fatal Virgin Galactic spac...
9076,6.260620e+17,#BBCLive Investigators say a fatal Virgin Gala...
9083,6.260630e+17,Investigators say a fatal Virgin Galactic spac...
9048,6.260650e+17,Investigators say a fatal Virgin Galactic spac...
1962,6.260760e+17,Schools in Western Uganda still Burning down B...
9047,6.260760e+17,Virgin galactic crash: early unlocking of brak...
9065,6.260760e+17,Virgin galactic crash: early unlocking of brak...
9049,6.260890e+17,Investigators rule catastrophic structural fai...
9089,6.260900e+17,NTSB: Virgin Galactic crash caused by structur...


i can see several points:


1.   Case #1 (based on 9083 and 9048 id): i can get difference values from `tweetids` field but same text apparently. 
2.   Case #2 : i can several duplicated value in `tweetids` field.
3.   Case #3 (based on 9076 and  9066) : i can get similar texts but with its differences from same value in `tweetids` field.

Based in these points i need to do following 🧰:


*   Remove URLs and arrobas.
*   Replace hashtag content with non-hashtag version.
*   Get the most long length version of text to cover more information.
*   Check the distribution of `choose one` field after that, to create a clean dataset (based in the ideas) and the label.


## Create a new and cleaning dataset